In [5]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [6]:
credenciais = json.load(open('auth.json'))
client_id = credenciais['client_id']
client_secret = credenciais['client_secret']

In [7]:
ccm = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=ccm)

In [ ]:
uri = 'spotify:artist:6XyY86QOPPrYVGvF9ch6wz'

linkin_park_data = sp.artist(uri)

albums = sp.artist_albums(linkin_park_data['id'], album_type='album')['items']

album_data = []

for album in albums:
  album_name = album['name']
  release_date = album['release_date']
  total_tracks = album['total_tracks']
  album_id = album['id']

  album_details = sp.album(album_id=album_id)
  popularity = album_details['popularity']

  album_data.append({
    'Nome Album': album_name,
    'Data de Lancamento': release_date,
    'Total de Faixas': total_tracks,
    'Popularidade': popularity
  })

albums_df = pd.DataFrame(album_data)


In [34]:
albums_df.to_csv('linkin_park_data.csv', index=False)

In [35]:
lp_df = pd.read_csv('linkin_park_data.csv')
lp_df

,Nome Album,Data de Lancamento,Total de Faixas,Popularidade
0,From Zero,2024-11-15,11,91
1,Papercuts: Instrumentals,2024-06-28,20,57
2,Papercuts,2024-04-12,20,67
3,Meteora 20th Anniversary Edition,2023-04-07,89,72
4,Hybrid Theory (20th Anniversary Edition),2020-10-09,80,61
5,One More Light Live,2017-12-15,16,56
6,One More Light,2017-05-19,10,73
7,The Hunting Party: Acapellas + Instrumentals,2014-06-09,22,37
8,The Hunting Party,2014-06-09,12,67
9,RECHARGED,2013-10-25,14,55
